In [1]:
import gensim
import numpy as np
import pandas as pd
from google.colab import drive
from gensim.models import KeyedVectors

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
uk_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/YaMlContest/embedding_w1/cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/YaMlContest/embedding_w1/cc.ru.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [9]:
uk_ru_train, X_train, Y_train = load_word_pairs("/content/drive/MyDrive/YaMlContest/embedding_w1/ukr_rus.train.txt")

In [10]:
uk_ru_test, X_test, Y_test = load_word_pairs("/content/drive/MyDrive/YaMlContest/embedding_w1/ukr_rus.test.txt")

Xi - это распределенное прелставление слова i на исходном языке, тогда Yi - это вектор слова на языке, который мы хотим перевести. Нам необходимо реализовать такое линейное преобразование W, чтобы минимизировать евклидова расстояние между WXi и Yi

In [11]:
from sklearn.linear_model import LinearRegression

In [14]:
mapping = LinearRegression().fit(X_train, Y_train)

In [69]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541592955589294),
 ('июнь', 0.8411963582038879),
 ('март', 0.8397400379180908),
 ('сентябрь', 0.8359215259552002),
 ('февраль', 0.8328748941421509),
 ('октябрь', 0.8311805725097656),
 ('ноябрь', 0.8278146982192993),
 ('июль', 0.8236350417137146),
 ('август', 0.8120611906051636),
 ('декабрь', 0.8037999272346497)]

In [57]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        for (f, s) in ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn):
            if ru == f:
                num_matches += 1

    precision_val = num_matches / len(pairs)
    print(precision_val)

    return precision_val

In [58]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

0.0
1.0
1.0


In [61]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.811

0.6356589147286822
0.8113695090439277


In [73]:
from numpy.linalg import svd

def learn_transform(X_train, Y_train):
    """
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """

    u, s, vh = svd(np.matmul(X_train.T, Y_train), full_matrices=True)
    return np.matmul(u, vh)

In [74]:
W = learn_transform(X_train, Y_train)

In [75]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237906694412231),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025653958320618),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [76]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

0.6537467700258398
0.8242894056847545


In [77]:
with open("/content/drive/MyDrive/YaMlContest/embedding_w1/fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [88]:
pip install nltk


In [91]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [140]:
from nltk.compat import DATA_UPDATES
from nltk.tokenize import word_tokenize

def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    tokenizer = word_tokenize(sentence.lower())
    data_trans = ''

    for i in tokenizer:
      if i in uk_emb.index_to_key:
        data_trans += ru_emb.most_similar(np.matmul(uk_emb[i], W), topn=1)[0][0]
        data_trans += " "
      else:
        data_trans += " "
    data_trans = data_trans.lstrip().rstrip()
    return data_trans


In [141]:
print(translate("кіт зловив мишу"))

кот поймал мышку


In [142]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [143]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк –

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала избушку , и и живет . а оно приходят морозы . из лисичка замерзла и и побежала во село огня  , чтобы  . прибегает к одной бабы и и говорит :

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые были , бабушку ! со воскресеньем ...  мне огня , мной тебе  .

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — хорошо , — говорит , —  – сестричка . садись  немного , пока мной пирожки  со печи !

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала сміттячка, стулила та й біжить.
